In [3]:
import os.path as osp

from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder
from latent_3d_points.src.tf_utils import reset_tf_graph

/home/prashant/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
experiment_name = 'single_class_sharp_ae'
train_dir =  '../data/single_class_sharp_ae'
n_pc_points = 32
bneck_size = 32                                # Bottleneck-AE size
ae_loss = 'emd' 
top_out_dir = '../data/'

In [12]:
from latent_3d_points.src.in_out import create_dir
train_dir = create_dir(osp.join(top_out_dir, experiment_name))
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)

In [13]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = 'single_class_sharp_ae'
conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).
conf.save(osp.join(train_dir, 'configuration'))

In [14]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

Building Encoder
('encoder_conv_layer_0', 'conv params = ', 128)
('bnorm params = ', 64)
Tensor("single_class_sharp_ae_2/Relu:0", shape=(?, 32, 32), dtype=float32)
('output size:', 1024, '\n')
('encoder_conv_layer_1', 'conv params = ', 2112)
('bnorm params = ', 128)
Tensor("single_class_sharp_ae_2/Relu_1:0", shape=(?, 32, 64), dtype=float32)
('output size:', 2048, '\n')
('encoder_conv_layer_2', 'conv params = ', 4160)
('bnorm params = ', 128)
Tensor("single_class_sharp_ae_2/Relu_2:0", shape=(?, 32, 64), dtype=float32)
('output size:', 2048, '\n')
('encoder_conv_layer_3', 'conv params = ', 8320)
('bnorm params = ', 256)
Tensor("single_class_sharp_ae_2/Relu_3:0", shape=(?, 32, 128), dtype=float32)
('output size:', 4096, '\n')
('encoder_conv_layer_4', 'conv params = ', 4128)
('bnorm params = ', 64)
Tensor("single_class_sharp_ae_2/Relu_4:0", shape=(?, 32, 32), dtype=float32)
('output size:', 1024, '\n')
Tensor("single_class_sharp_ae_2/Max:0", shape=(?, 32), dtype=float32)
Building Decoder


In [15]:
training_file = '/home/prashant/Desktop/training_corners.xyzs'
#training_file = '/home/prashant/Desktop/fandisk_flagged.xyzs'

import numpy as np

xyzs = np.loadtxt(training_file).astype(np.float32)

xyzs = xyzs[xyzs[:,3] >= 0.5]

xyz = xyzs[:,:3]
sharps = xyzs[:,3, np.newaxis]

num_points = xyz.shape[0]

print('Number of points = ' , num_points)

from scipy.spatial import KDTree
kdtree = KDTree(xyz)

from multiprocessing import Pool
import time
from latent_3d_points.src.in_out import PointCloudDataSet


def get_neighbors(index):
    selected_point = xyz[index]
    neighb_dists, neighb_indices = kdtree.query(selected_point, n_pc_points)
    return xyz[neighb_indices]

start_time = time.time()

pool = Pool(processes=8)

pclouds = np.array(pool.map(get_neighbors, range(num_points)))

pool.close()
pool.join()

all_pc_data = PointCloudDataSet(pclouds, labels=None, init_shuffle=False)


end_time = time.time()
kd_duration = end_time - start_time

print('Preparing all neighborhoods took %d seconds' % kd_duration)

buf_size = 1 # flush each line
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(all_pc_data, conf, log_file=fout)
fout.close()

Number of points =  2250
Preparing all neighborhoods took 1 seconds
('Epoch:', '0001', 'training time (minutes)=', '0.0065', 'loss=', '0.751776928')
INFO:tensorflow:../data/single_class_sharp_ae/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.0032', 'loss=', '0.374510620')
('Epoch:', '0003', 'training time (minutes)=', '0.0030', 'loss=', '0.217956086')
('Epoch:', '0004', 'training time (minutes)=', '0.0031', 'loss=', '0.151344191')
('Epoch:', '0005', 'training time (minutes)=', '0.0031', 'loss=', '0.121890658')
('Epoch:', '0006', 'training time (minutes)=', '0.0031', 'loss=', '0.111093845')
('Epoch:', '0007', 'training time (minutes)=', '0.0030', 'loss=', '0.105684660')
('Epoch:', '0008', 'training time (minutes)=', '0.0031', 'loss=', '0.102935088')
('Epoch:', '0009', 'training time (minutes)=', '0.0030', 'loss=', '0.093789088')
('Epoch:', '0010', 'training time (minutes)=', '0.0031', 'loss=', '0.097413588')
INFO

('Epoch:', '0089', 'training time (minutes)=', '0.0030', 'loss=', '0.053886962')
('Epoch:', '0090', 'training time (minutes)=', '0.0030', 'loss=', '0.056429921')
INFO:tensorflow:../data/single_class_sharp_ae/models.ckpt-90 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0091', 'training time (minutes)=', '0.0031', 'loss=', '0.051958262')
('Epoch:', '0092', 'training time (minutes)=', '0.0030', 'loss=', '0.054553230')
('Epoch:', '0093', 'training time (minutes)=', '0.0031', 'loss=', '0.053637116')
('Epoch:', '0094', 'training time (minutes)=', '0.0030', 'loss=', '0.053192788')
('Epoch:', '0095', 'training time (minutes)=', '0.0029', 'loss=', '0.054441723')
('Epoch:', '0096', 'training time (minutes)=', '0.0030', 'loss=', '0.052493386')
('Epoch:', '0097', 'training time (minutes)=', '0.0029', 'loss=', '0.058284346')
('Epoch:', '0098', 'training time (minutes)=', '0.0030', 'loss=', '0.053953475')
('Epoch:', '0099', 'training time (minutes)=', '0.0030', 'loss=', '0.05

('Epoch:', '0177', 'training time (minutes)=', '0.0038', 'loss=', '0.044419702')
('Epoch:', '0178', 'training time (minutes)=', '0.0030', 'loss=', '0.044667550')
('Epoch:', '0179', 'training time (minutes)=', '0.0030', 'loss=', '0.043411637')
('Epoch:', '0180', 'training time (minutes)=', '0.0030', 'loss=', '0.044016270')
INFO:tensorflow:../data/single_class_sharp_ae/models.ckpt-180 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0181', 'training time (minutes)=', '0.0040', 'loss=', '0.044148489')
('Epoch:', '0182', 'training time (minutes)=', '0.0030', 'loss=', '0.043289243')
('Epoch:', '0183', 'training time (minutes)=', '0.0030', 'loss=', '0.046265966')
('Epoch:', '0184', 'training time (minutes)=', '0.0039', 'loss=', '0.042530546')
('Epoch:', '0185', 'training time (minutes)=', '0.0034', 'loss=', '0.045436836')
('Epoch:', '0186', 'training time (minutes)=', '0.0031', 'loss=', '0.046150704')
('Epoch:', '0187', 'training time (minutes)=', '0.0039', 'loss=', '0.0

('Epoch:', '0265', 'training time (minutes)=', '0.0040', 'loss=', '0.040198103')
('Epoch:', '0266', 'training time (minutes)=', '0.0031', 'loss=', '0.039234343')
('Epoch:', '0267', 'training time (minutes)=', '0.0033', 'loss=', '0.039455679')
('Epoch:', '0268', 'training time (minutes)=', '0.0039', 'loss=', '0.038822471')
('Epoch:', '0269', 'training time (minutes)=', '0.0032', 'loss=', '0.038509390')
('Epoch:', '0270', 'training time (minutes)=', '0.0033', 'loss=', '0.037448080')
INFO:tensorflow:../data/single_class_sharp_ae/models.ckpt-270 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0271', 'training time (minutes)=', '0.0037', 'loss=', '0.037516847')
('Epoch:', '0272', 'training time (minutes)=', '0.0030', 'loss=', '0.035923918')
('Epoch:', '0273', 'training time (minutes)=', '0.0035', 'loss=', '0.038082670')
('Epoch:', '0274', 'training time (minutes)=', '0.0031', 'loss=', '0.037929961')
('Epoch:', '0275', 'training time (minutes)=', '0.0039', 'loss=', '0.0

('Epoch:', '0353', 'training time (minutes)=', '0.0037', 'loss=', '0.036208080')
('Epoch:', '0354', 'training time (minutes)=', '0.0042', 'loss=', '0.035595158')
('Epoch:', '0355', 'training time (minutes)=', '0.0047', 'loss=', '0.035039472')
('Epoch:', '0356', 'training time (minutes)=', '0.0042', 'loss=', '0.037764620')
('Epoch:', '0357', 'training time (minutes)=', '0.0034', 'loss=', '0.036061615')
('Epoch:', '0358', 'training time (minutes)=', '0.0033', 'loss=', '0.035026250')
('Epoch:', '0359', 'training time (minutes)=', '0.0044', 'loss=', '0.034963518')
('Epoch:', '0360', 'training time (minutes)=', '0.0042', 'loss=', '0.036213469')
INFO:tensorflow:../data/single_class_sharp_ae/models.ckpt-360 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0361', 'training time (minutes)=', '0.0041', 'loss=', '0.033708245')
('Epoch:', '0362', 'training time (minutes)=', '0.0054', 'loss=', '0.034357416')
('Epoch:', '0363', 'training time (minutes)=', '0.0036', 'loss=', '0.0

('Epoch:', '0441', 'training time (minutes)=', '0.0037', 'loss=', '0.032924630')
('Epoch:', '0442', 'training time (minutes)=', '0.0038', 'loss=', '0.033586726')
('Epoch:', '0443', 'training time (minutes)=', '0.0039', 'loss=', '0.033626027')
('Epoch:', '0444', 'training time (minutes)=', '0.0034', 'loss=', '0.033883549')
('Epoch:', '0445', 'training time (minutes)=', '0.0036', 'loss=', '0.032921284')
('Epoch:', '0446', 'training time (minutes)=', '0.0030', 'loss=', '0.030938656')
('Epoch:', '0447', 'training time (minutes)=', '0.0037', 'loss=', '0.032564115')
('Epoch:', '0448', 'training time (minutes)=', '0.0031', 'loss=', '0.034585089')
('Epoch:', '0449', 'training time (minutes)=', '0.0030', 'loss=', '0.033030453')
('Epoch:', '0450', 'training time (minutes)=', '0.0029', 'loss=', '0.032213694')
INFO:tensorflow:../data/single_class_sharp_ae/models.ckpt-450 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0451', 'training time (minutes)=', '0.0030', 'loss=', '0.0

In [16]:
conf = Conf.load(osp.join(train_dir, 'configuration'))

In [17]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

Building Encoder
('encoder_conv_layer_0', 'conv params = ', 128)
('bnorm params = ', 64)
Tensor("single_class_sharp_ae_2/Relu:0", shape=(?, 32, 32), dtype=float32)
('output size:', 1024, '\n')
('encoder_conv_layer_1', 'conv params = ', 2112)
('bnorm params = ', 128)
Tensor("single_class_sharp_ae_2/Relu_1:0", shape=(?, 32, 64), dtype=float32)
('output size:', 2048, '\n')
('encoder_conv_layer_2', 'conv params = ', 4160)
('bnorm params = ', 128)
Tensor("single_class_sharp_ae_2/Relu_2:0", shape=(?, 32, 64), dtype=float32)
('output size:', 2048, '\n')
('encoder_conv_layer_3', 'conv params = ', 8320)
('bnorm params = ', 256)
Tensor("single_class_sharp_ae_2/Relu_3:0", shape=(?, 32, 128), dtype=float32)
('output size:', 4096, '\n')
('encoder_conv_layer_4', 'conv params = ', 4128)
('bnorm params = ', 64)
Tensor("single_class_sharp_ae_2/Relu_4:0", shape=(?, 32, 32), dtype=float32)
('output size:', 1024, '\n')
Tensor("single_class_sharp_ae_2/Max:0", shape=(?, 32), dtype=float32)
Building Decoder


In [18]:
ae.restore_model(train_dir, 500)

INFO:tensorflow:Restoring parameters from ../data/single_class_sharp_ae/models.ckpt-500


In [19]:
import tensorflow as tf
from latent_3d_points.src.encoders_decoders import decoder_with_fc_only

In [20]:
with tf.variable_scope("transfer"):
    new_decoder = decoder_with_fc_only(ae.z, [64, 64, 1], verbose=True, b_norm=False)

Building Decoder
('decoder_fc_0', 'FC params = ', 2112) Tensor("transfer/Relu:0", shape=(?, 64), dtype=float32)
('output size:', 64, '\n')
('decoder_fc_1', 'FC params = ', 4160) Tensor("transfer/Relu_1:0", shape=(?, 64), dtype=float32)
('output size:', 64, '\n')
('decoder_fc_2', 'FC params = ', 65) Tensor("transfer/decoder_fc_2/BiasAdd:0", shape=(?, 1), dtype=float32)
('output size:', 1, '\n')


In [21]:
training_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "transfer")
training_vars

[<tf.Variable 'transfer/decoder_fc_0/W:0' shape=(32, 64) dtype=float32_ref>,
 <tf.Variable 'transfer/decoder_fc_0/b:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'transfer/decoder_fc_1/W:0' shape=(64, 64) dtype=float32_ref>,
 <tf.Variable 'transfer/decoder_fc_1/b:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'transfer/decoder_fc_2/W:0' shape=(64, 1) dtype=float32_ref>,
 <tf.Variable 'transfer/decoder_fc_2/b:0' shape=(1,) dtype=float32_ref>]

In [22]:
learning_rate = 0.0001
optimizer = tf.train.AdamOptimizer(learning_rate)
sharpness_target_var = tf.placeholder(tf.float32, shape=(None, 1), name='sharpness_target')
cost = tf.losses.mean_squared_error(sharpness_target_var, new_decoder)
train_op = optimizer.minimize(cost, var_list=training_vars)

In [23]:
init_new_vars_op = tf.variables_initializer(training_vars)
optimizer_initializers = [var.initializer for var in tf.global_variables() if 'Adam' in var.name or 'beta' in var.name]
ae.sess.run(init_new_vars_op)
ae.sess.run(optimizer_initializers)
print('Any variables listed here have not been initialized:')
ae.sess.run(tf.report_uninitialized_variables(tf.global_variables()))

Any variables listed here have not been initialized:


array([], dtype=object)

In [30]:
num_epochs = 200

training_file = '/home/prashant/Desktop/training_corners.xyzs'
#training_file = '/home/prashant/Desktop/fandisk_flagged.xyzs'

num_points_input = n_pc_points
batch_size = 32

import numpy as np

xyzs = np.loadtxt(training_file).astype(np.float32)

xyz = xyzs[:,:3]
sharps = xyzs[:,3, np.newaxis]

num_points = xyz.shape[0]
num_batches = (num_points + batch_size - 1) // batch_size

print('Batch size = ' , batch_size)
print('Number of points = ' , num_points)
print('Number of batches = ' , num_batches)


from scipy.spatial import KDTree
kdtree = KDTree(xyz)

from multiprocessing import Pool
import time
from latent_3d_points.src.in_out import PointCloudDataSet


def get_neighbors(index):
    selected_point = xyz[index]
    neighb_dists, neighb_indices = kdtree.query(selected_point, n_pc_points)
    return xyz[neighb_indices]

start_time = time.time()

pool = Pool(processes=8)

pclouds = np.array(pool.map(get_neighbors, range(num_points)))

pool.close()
pool.join()

Batch size =  32
Number of points =  12326
Number of batches =  386


In [31]:
from multiprocessing import Pool, cpu_count
cpu_count()

8

In [32]:
from multiprocessing import Pool
from tflearn import is_training
import time


def get_neighbors(index):
    selected_point = xyz[index]
    neighb_dists, neighb_indices = kdtree.query(selected_point, num_points_input)
    return xyz[neighb_indices]

ae.sess.run(init_new_vars_op)
ae.sess.run(optimizer_initializers)

is_training(True, session=ae.sess)

start_time = time.time()

for epoch_i in range(num_epochs):
    
    #print('Epoch ', epoch_i)
    all_indices = np.random.permutation(num_points)
    epochLoss = 0.0
    
    for batch_i in range(num_batches):
                
        batchStart = batch_i * batch_size
        batchEnd = min(batchStart + batch_size, num_points)
        chosenIndices = all_indices[batchStart:batchEnd]
        chosenSharps = sharps[chosenIndices]
        chosenPoints = xyz[chosenIndices]
        
        chosenNeighbors = pclouds[chosenIndices]
        
        #pool = Pool(processes=8)
        
        #chosenNeighbors = np.array(pool.map(get_neighbors, chosenIndices))
        
        #pool.close()
        #pool.join()
        
        _, loss = ae.sess.run((train_op, cost), 
                              feed_dict={ae.x: chosenNeighbors, sharpness_target_var: chosenSharps})
        
        #print(("Epoch:", '%04d' % (epoch_i), 
        #       "Batch:", '%04d' % (batch_i),
        #        "loss=", "{:.9f}".format(loss)))
        
        epochLoss += loss
    
    epochLoss /= num_batches
    
    print(("Epoch:", '%04d' % (epoch_i),
                "loss=", "{:.9f}".format(epochLoss)))
    
end_time = time.time()

print('Training finished in %d seconds ' % (end_time - start_time))

    

('Epoch:', '0000', 'loss=', '0.124515668')
('Epoch:', '0001', 'loss=', '0.097742842')
('Epoch:', '0002', 'loss=', '0.094328382')
('Epoch:', '0003', 'loss=', '0.091467811')
('Epoch:', '0004', 'loss=', '0.090872955')
('Epoch:', '0005', 'loss=', '0.088412619')
('Epoch:', '0006', 'loss=', '0.088815538')
('Epoch:', '0007', 'loss=', '0.087472610')
('Epoch:', '0008', 'loss=', '0.085803676')
('Epoch:', '0009', 'loss=', '0.084983210')
('Epoch:', '0010', 'loss=', '0.083772626')
('Epoch:', '0011', 'loss=', '0.083946554')
('Epoch:', '0012', 'loss=', '0.084381245')
('Epoch:', '0013', 'loss=', '0.084056231')
('Epoch:', '0014', 'loss=', '0.083136971')
('Epoch:', '0015', 'loss=', '0.082392763')
('Epoch:', '0016', 'loss=', '0.083120148')
('Epoch:', '0017', 'loss=', '0.082360413')
('Epoch:', '0018', 'loss=', '0.081857701')
('Epoch:', '0019', 'loss=', '0.081252896')
('Epoch:', '0020', 'loss=', '0.080382927')
('Epoch:', '0021', 'loss=', '0.080750465')
('Epoch:', '0022', 'loss=', '0.081379016')
('Epoch:', 

('Epoch:', '0191', 'loss=', '0.067162501')
('Epoch:', '0192', 'loss=', '0.067732205')
('Epoch:', '0193', 'loss=', '0.068285701')
('Epoch:', '0194', 'loss=', '0.067956613')
('Epoch:', '0195', 'loss=', '0.068189945')
('Epoch:', '0196', 'loss=', '0.068064032')
('Epoch:', '0197', 'loss=', '0.068672407')
('Epoch:', '0198', 'loss=', '0.068164060')
('Epoch:', '0199', 'loss=', '0.068102586')
Training finished in 99 seconds 


In [33]:
all_indices = np.arange(num_points)
result = np.zeros((0,1))

is_training(False, session=ae.sess)

for batch_i in range(num_batches):
                
    batchStart = batch_i * batch_size
    batchEnd = min(batchStart + batch_size, num_points)
    chosenIndices = all_indices[batchStart:batchEnd]
    chosenSharps = sharps[chosenIndices]
    chosenPoints = xyz[chosenIndices]

    chosenNeighbors = pclouds[chosenIndices]

        #pool = Pool(processes=8)

        #chosenNeighbors = np.array(pool.map(get_neighbors, chosenIndices))

        #pool.close()
        #pool.join()
    pred = ae.sess.run((new_decoder), 
                          feed_dict={ae.x: chosenNeighbors})
    result = np.append(result, pred, axis=0)
    
np.savetxt('sharp_result.txt', result)

print('Result saved')

Result saved
